In [ ]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import xarray as xr
import matplotlib.colors as pltc
import geopandas
import datetime as dt
from scipy import stats
from sklearn import preprocessing
import s3fs
import sys, os, glob,re
import multiprocessing as mp
import time as time


## Cubic feet to cubic meters conversion factor
cfs_2_cms = 0.0283168466

In [2]:
pnwNP = pd.read_csv("../data/pnwNP_Info.csv")

### Pull out sites
shp = geopandas.read_file("../data/VIC_UW/shapefiles/columbia_seg.shp")
shp = pd.merge(pnwNP,shp,left_on='comid',right_on='POI_ID').fillna(0)
seg_ids = np.array(shp['seg_id'])

In [3]:
## Open modeled datasets and extract data at segment locations.
pnwVIC = xr.open_mfdataset('../data/VIC_UW/pnwNP_Gages_VIC.nc')
pnwVIC = pnwVIC.where(pnwVIC['reachID'].isin(seg_ids),drop=True)

pnwPRMS = xr.open_mfdataset('../data/VIC_UW/pnwNP_Gages_PRMS.nc')
pnwPRMS = pnwPRMS.where(pnwPRMS['reachID'].isin(seg_ids),drop=True)


## Open NWM from NOAA AWS bucket
s3_path = 's3://noaa-nwm-retro-v2-zarr-pds' #nwm 2.0

# Connect to S3
s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root=s3_path, s3=s3, check=False)

# load the dataset
ds = xr.open_zarr(store=store, consolidated=True)


In [4]:
for i in range(len(shp)):
    print(shp['gage'][i])
    fields = ['X_00060_00003', 'Date']

    ## Get Vic data
    VIC = pnwVIC.where(pnwVIC['reachID']==shp['seg_id'][i],drop=True).to_dataframe()
    VIC = VIC.drop(['reachID'],axis=1)
    VIC= VIC.droplevel('seg')
    VIC['time'] = pd.to_datetime(VIC.index,)
    VIC['time'] = VIC['time'].dt.tz_localize(None)
    VIC = VIC.reset_index(drop=True)
    VIC.columns = ["streamflow_VIC","time"]



    ## Get PRMS data
    PRMS = pnwPRMS.where(pnwPRMS['reachID']==shp['seg_id'][i],drop=True).to_dataframe()
    PRMS = PRMS.drop(['reachID'],axis=1)
    PRMS= PRMS.droplevel('seg')
    PRMS['time'] = pd.to_datetime(PRMS.index,)
    PRMS['time'] = PRMS['time'].dt.tz_localize(None)
    PRMS = PRMS.reset_index(drop=True)
    PRMS.columns = ["streamflow_PRMS","time"]


    ## Combine
    datMain = pd.merge(VIC,PRMS, on='time',how='inner')

    ## Get NWM 2.0 data
    # slice all data using a specific reach identifier and time range
    timerange = slice('1979-01-31', '2011-12-31')
    dat = ds.sel(feature_id=shp['comid'][i],
                 time=timerange).streamflow.persist() 

    # This step takes a bit longer because it's actually returning the data
    dat = dat.resample(time='1d').mean()

    NWM = pd.DataFrame(dat.to_pandas())
    NWM['time'] = pd.to_datetime(NWM.index)
    NWM.columns = ["streamflow_NWM2d0","time"]
    NWM = NWM.reset_index(drop=True)
    NWM["gage"] = shp['gage'][i]


    ## Combine with above datasets
    datMain = pd.merge(datMain,NWM, on='time',how='inner')

    # ## Get NWIS Data
    path  = "../data/NWIS_streamflow/"
    gage = '*'+str(shp['gage'][i])+'*'
    file = glob.glob("../data/NWIS_streamflow/"+gage)[0]

    NWIS = pd.read_csv(file,usecols=fields)
    NWIS.columns = ["time","streamflow_NWIS"]
    NWIS['time'] = pd.to_datetime(NWIS['time']).dt.tz_localize(None)
    NWIS["streamflow_NWIS"] = NWIS["streamflow_NWIS"]*cfs_2_cms

    ## Combine all the data
    datMain = pd.merge(datMain,NWIS, on='time',how='inner')

    datMain.to_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+".csv")

12115700
12116500
12323770
12465000
12513000
13055340
13058000
13068500
13068501
13082500
13112000
13132500
13132520
13132535
13142500
13152500
13159800
13174500
13215000
13217500
13236500
13297350
14015000
14018500
14034470
14034500
14233500
14362250


### Function

In [ ]:
def extractModel():
    print(shp['gage'])
    fields = ['X_00060_00003', 'Date']

    ## Get Vic data
    VIC = pnwVIC.where(pnwVIC['reachID']==shp['seg_id'],drop=True).to_dataframe()
    VIC = VIC.drop(['reachID'],axis=1)
    VIC= VIC.droplevel('seg')
    VIC['time'] = pd.to_datetime(VIC.index,)
    VIC['time'] = VIC['time'].dt.tz_localize(None)
    VIC = VIC.reset_index(drop=True)
    VIC.columns = ["streamflow_VIC","time"]



    ## Get PRMS data
    PRMS = pnwPRMS.where(pnwPRMS['reachID']==shp['seg_id'],drop=True).to_dataframe()
    PRMS = PRMS.drop(['reachID'],axis=1)
    PRMS= PRMS.droplevel('seg')
    PRMS['time'] = pd.to_datetime(PRMS.index,)
    PRMS['time'] = PRMS['time'].dt.tz_localize(None)
    PRMS = PRMS.reset_index(drop=True)
    PRMS.columns = ["streamflow_PRMS","time"]


    ## Combine
    datMain = pd.merge(VIC,PRMS, on='time',how='inner')

    ## Get NWM 2.0 data
    # slice all data using a specific reach identifier and time range
    timerange = slice('1979-01-31', '2011-12-31')
    dat = ds.sel(feature_id=shp['comid'],
                 time=timerange).streamflow.persist() 

    # This step takes a bit longer because it's actually returning the data
    dat = dat.resample(time='1d').mean()

    NWM = pd.DataFrame(dat.to_pandas())
    NWM['time'] = pd.to_datetime(NWM.index)
    NWM.columns = ["streamflow_NWM2d0","time"]
    NWM = NWM.reset_index(drop=True)
    NWM["gage"] = shp['gage']


    ## Combine with above datasets
    datMain = pd.merge(datMain,NWM, on='time',how='inner')

    # ## Get NWIS Data
    path  = "../data/NWIS_streamflow/"
    gage = '*'+str(shp['gage'])+'*'
    file = glob.glob("../data/NWIS_streamflow/"+gage)[0]

    NWIS = pd.read_csv(file,usecols=fields)
    NWIS.columns = ["time","streamflow_NWIS"]
    NWIS['time'] = pd.to_datetime(NWIS['time']).dt.tz_localize(None)
    NWIS["streamflow_NWIS"] = NWIS["streamflow_NWIS"]*cfs_2_cms

    ## Combine all the data
    datMain = pd.merge(datMain,NWIS, on='time',how='inner')

    datMain.to_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+".csv")


In [ ]:
def pnwModeledDatamp():
    print(shp['gage'])
    # Make a timer for the processes
    start = time.process_time()
    
    # Step 1: Init multiprocessing.Pool()
    pool = mp.Pool(mp.cpu_count()-1)

    # Step 2: `pool.starmap` the `buildVTK()` function
    results = pool.starmap(extractModel,iterable=shp)

    # Step 3: Don't forget to close
    pool.close()
    
    #Finish timing
    end = time.process_time()
    print(end - start)